# Introducción
En este notebook tenemos el código para hacer un gráfico y distintos indicadores para realizar un análisis técnico

In [22]:
# Librerias
import numpy as np
import pandas as pd
import streamlit as st
# Stock
import yfinance as yf
from lightweight_charts import Chart, JupyterChart

In [3]:
# Descargamos los datos de apple mismo
start_date = "2024-01-01"
end_date = "2024-12-12"
interval='1h'

In [5]:
df = yf.download("TSLA", start=start_date, end=end_date, interval=interval)
df

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA
Datetime,,,,,,
2024-01-02 14:30:00+00:00,249.610001,249.610001,251.250000,244.410004,250.000000,36427280
2024-01-02 15:30:00+00:00,250.000000,250.000000,250.259995,246.820007,249.600006,16932853
2024-01-02 16:30:00+00:00,248.350006,248.350006,251.000000,247.941696,249.994095,12121725
2024-01-02 17:30:00+00:00,248.351700,248.351700,249.132996,247.000000,248.354996,9300163
2024-01-02 18:30:00+00:00,248.137207,248.137207,249.380005,247.300003,248.389999,8770028
...,...,...,...,...,...,...
2024-12-11 16:30:00+00:00,413.434998,413.434998,415.000000,410.850006,412.589996,10219649
2024-12-11 17:30:00+00:00,417.359406,417.359406,417.850006,412.410095,413.399994,10949668


In [11]:
# RSI
def calculate_rsi(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    delta = df['close'].diff()  # Calcula los cambios entre precios consecutivos
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()  # Promedio de ganancias
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()  # Promedio de pérdidas

    rs = gain / loss  # Relative Strength
    rsi = 100 - (100 / (1 + rs))  # RSI fórmula

    return pd.DataFrame(
        {
            'RSI': rsi
        }
    ).dropna()

In [12]:
calculate_rsi(df=df, period=14)

,RSI
Datetime,
2024-01-03 20:30:00+00:00,25.063519
2024-01-04 14:30:00+00:00,31.351080
2024-01-04 15:30:00+00:00,33.732063
2024-01-04 16:30:00+00:00,35.268049
2024-01-04 17:30:00+00:00,35.219989
...,...
2024-12-11 16:30:00+00:00,78.087743
2024-12-11 17:30:00+00:00,77.670914
2024-12-11 18:30:00+00:00,79.869884


In [8]:
# Limpiamos el multiindex
rename_columns = {'Open' : 'open', 
                  'High':  'high', 
                  'Low' : 'low', 
                  'Close' : 'close', 
                  'Adj Close' : 'adj close', 
                  'Volume' : 'volume'}
df.rename(columns=rename_columns, inplace=True)
df.columns = df.columns.droplevel('Ticker')

In [20]:
def get_bar_data(symbol, start_date, end_date, interval):
    df = yf.download(symbol, start=start_date, end=end_date, interval=interval)
    # Limpiamos el multiindex
    rename_columns = {'Open' : 'open', 
                    'High':  'high', 
                    'Low' : 'low', 
                    'Close' : 'close', 
                    'Adj Close' : 'adj close', 
                    'Volume' : 'volume'}
    df.rename(columns=rename_columns, inplace=True)
    df.columns = df.columns.droplevel('Ticker')
    return df

def on_search(chart, searched_string):  # Called when the user searches.
    new_data = get_bar_data(searched_string, chart.topbar['timeframe'].value)
    if new_data.empty:
        return
    chart.topbar['symbol'].set(searched_string)
    chart.set(new_data)


def on_timeframe_selection(chart):  # Called when the user changes the timeframe.
    new_data = get_bar_data(
        symbol=chart.topbar['symbol'].value,
        start_date= start_date,
        interval=chart.topbar['timeframe'].value)
    if new_data.empty:
        return
    chart.set(new_data, True)

In [13]:
# RSI
def calculate_rsi(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    delta = df['close'].diff()  # Calcula los cambios entre precios consecutivos
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()  # Promedio de ganancias
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()  # Promedio de pérdidas

    rs = gain / loss  # Relative Strength
    rsi = 100 - (100 / (1 + rs))  # RSI fórmula

    return pd.DataFrame(
        {
            #'time': df.index,
            'RSI': rsi
        }
    ).dropna()

In [29]:
# Instanciamos el gráfico
chart = Chart(toolbox=True)
chart2 = chart.create_subchart(position='left', width=1, height=0.2, sync=True)
chart.watermark('1')
chart2.watermark('2')
# line = chart2.create_line()
# chart.events.search += on_search
# chart.topbar.textbox('symbol', 'TSLA')

# chart.topbar.switcher(
#     'timeframe', ('1min', '5min', '1d'),
#     func=on_timeframe_selection
# )
df = get_bar_data('TSLA', start_date, end_date, interval)
rsi =  calculate_rsi(df=df, period=14)

chart.set(df)
chart2.set(df)
# line.set(rsi)
# chart.horizontal_line(200, func=on_horizontal_line_move)

chart.show(block=True)

[*********************100%***********************]  1 of 1 completed


# Usamos streamlit

In [25]:
rsi

,RSI
Datetime,
2024-01-03 20:30:00+00:00,25.063519
2024-01-04 14:30:00+00:00,31.351080
2024-01-04 15:30:00+00:00,33.732063
2024-01-04 16:30:00+00:00,35.268049
2024-01-04 17:30:00+00:00,35.219989
...,...
2024-12-11 16:30:00+00:00,78.087743
2024-12-11 17:30:00+00:00,77.670914
2024-12-11 18:30:00+00:00,79.869884
